In [1]:
import numpy as np
import matplotlib.pyplot as plt
import hashlib
import networkx as nx
import random
import math
import d3rlpy

import utils

/vol/bitbucket/phl23/targeted/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def channelfirst_for_d3rlpy(arr):
    return np.transpose(arr, (2, 0, 1))

def get_hash(s):
    flattened_obs = s.flatten()
    flattened_obs_bytes = flattened_obs.tobytes()   
    obs_hash = hashlib.sha256(flattened_obs_bytes).hexdigest()
    return obs_hash

In [3]:
def get_experience(env, model_path, seed, episodes=10, argmax=True, memory=False, text=False):
    utils.seed(seed)
    # Load environment
    env = utils.make_env(env, seed, render_mode="human")
    print("Environment loaded\n")

    # Load agent
    env.action_space.n = 3
    model_dir = utils.get_model_dir(model_path)
    agent = utils.Agent(env.observation_space, env.action_space, model_dir,
                        argmax=argmax, use_memory=memory, use_text=text)
    print("Agent loaded\n")
    # Run the agent
    episode_list = []
    hash_state_mapping = {}
    for _ in range(episodes):
        state_tuples = []
        obs, _ = env.reset()
        count = 0
        while True:
            current_tuple = []
            current_tuple.append(env.hash())
            if env.hash() not in hash_state_mapping.keys():
                hash_state_mapping[env.hash()] = channelfirst_for_d3rlpy(obs['image'])
            action = agent.get_action(obs)
            obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated | truncated
            agent.analyze_feedback(reward, done)
            count += 1
            current_tuple.extend([action, reward, env.hash(), done])
            if env.hash() not in hash_state_mapping.keys():
                hash_state_mapping[env.hash()] = channelfirst_for_d3rlpy(obs['image'])
            state_tuples.append(current_tuple)

            if done:
                break
        episode_list.append(state_tuples)
    return episode_list, hash_state_mapping

In [4]:
def build_graph(dataset):    
    exp_graph = nx.DiGraph()
    for exp in dataset:
        count = 0
        for s1, a, r, s2, done in exp:
            exp_graph.add_node(s1)
            exp_graph.add_node(s2)
            exp_graph.add_edges_from([(s1, s2, {'action': a})])
            count += 1
            
    return exp_graph

In [5]:
ENVIRONMENT = 'MiniGrid-Empty-Random-6x6-v0'
SEED = 1
MODEL_PATH = 'Empty6x6RandomPPO'
BUDGET = 3
PERCENTAGE = 5
POISONED_MODEL_PATH = './poisoned_minigrid_cql.d3'

In [6]:
experience_list, hash_state_mapping = get_experience(ENVIRONMENT, MODEL_PATH, SEED, episodes=1000)

Environment loaded

Agent loaded



/vol/bitbucket/phl23/targeted/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.hash to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.hash` for environment variables or `env.get_wrapper_attr('hash')` that will search the reminding wrappers.
  logger.warn(


In [7]:
graph = build_graph(experience_list)

In [8]:
def get_path_to_state(graph, start_state, end_state):
    try:
        path = nx.shortest_path(graph, source=start_state, target=end_state)
        return path
    except nx.NetworkXNoPath:
        print(f"No path found from {start_state} to {end_state}")
        return None
    
def get_actions_to_state(graph, path):
    edges_in_path = list(zip(path[:-1], path[1:]))
    edge = [graph[u][v]['action'] for u, v in edges_in_path]
    return edge

In [9]:
start_node = '9fe3d6c4d1261a84'
goal_node = 'b145243c6e3378f5'
path = get_path_to_state(graph, start_node, goal_node)
if path:
    print(get_actions_to_state(graph, path))

[array([0]), array([2]), array([2]), array([2]), array([0]), array([2]), array([2]), array([0]), array([0])]


In [10]:
def build_MDP_dataset(episode_list, hash_state_mapping):
    episodes = []
    for epi in episode_list:
        obs_list = []
        act_list = []
        reward_list = []
        terminate_list = []
        for s1, a, r, s2, info in epi:
            s1_obs = hash_state_mapping[s1]
            obs_list.append(s1_obs)
            act_list.append(a)
            reward_list.append(r)
            if info:
                terminate_list.append(1.0)
            else:
                terminate_list.append(0.0)

        obs_list = np.array(obs_list)
        act_list = np.array(act_list)
        reward_list = np.array(reward_list).reshape(-1, 1)
        terminate_list = np.array(terminate_list)

        episode = d3rlpy.dataset.Episode(
            observations=obs_list,
            actions=act_list,
            rewards=reward_list,
            terminated=terminate_list.any(),
        )

        episodes.append(episode)

    dataset = d3rlpy.dataset.ReplayBuffer(
        d3rlpy.dataset.InfiniteBuffer(),
        episodes=episodes,
        action_space=d3rlpy.ActionSpace.DISCRETE,
    )
    return dataset

In [11]:
clean_dataset = build_MDP_dataset(experience_list, hash_state_mapping)

2024-07-08 13:19.28 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('int64')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('uint8')], shape=[(3, 7, 7)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2024-07-08 13:19.28 [info     ] Action size has been automatically determined. action_size=3


In [12]:
import random
hash_keys = list(hash_state_mapping.keys())
print(len(hash_keys))
random_idx = random.sample(hash_keys, 30)
save = []
for i in random_idx:
    start_node = i
    goal_node = 'b145243c6e3378f5'
    path = get_path_to_state(graph, start_node, goal_node)
    if path:
        print(get_actions_to_state(graph, path))




62
[array([2]), array([2]), array([0]), array([0])]
[array([1]), array([2]), array([1]), array([2]), array([0]), array([2]), array([1])]
[array([2]), array([0]), array([2]), array([1])]
[array([0]), array([0]), array([2]), array([2]), array([2]), array([1])]
[array([1]), array([2]), array([1]), array([2]), array([2]), array([2]), array([1])]
[array([2]), array([2]), array([1]), array([2]), array([1])]
[array([1]), array([2]), array([2]), array([1]), array([2]), array([0]), array([2]), array([1])]
[array([1]), array([1]), array([2]), array([1]), array([2]), array([1])]
[array([0]), array([2]), array([0]), array([2]), array([2]), array([1])]
[array([0]), array([2]), array([1])]
[array([0]), array([2]), array([2]), array([0]), array([2]), array([2]), array([0]), array([0])]
[array([0]), array([0]), array([2]), array([2]), array([1]), array([2]), array([0]), array([2]), array([1])]
No path found from 141f54d6e37ef2a8 to b145243c6e3378f5
[array([2]), array([1]), array([2]), array([1])]
[]
[

In [13]:
print(random_idx)

['e44112fc5dd98da9', 'f0613f6993e0a30e', 'c5a46bf308a3ff92', '696334c442253f6f', '190e48fed297889f', '8e7f7ce5578f9df0', '1265d2b6592c95e6', '30751990dcd82e4f', '1fc221ae7c965c16', '7c1df098ce3b9041', 'd6a4e14cb3418531', 'e9407b4add60085d', '141f54d6e37ef2a8', '6692c18231ad0423', 'b145243c6e3378f5', '2404c28dbd3255c7', '9fe3d6c4d1261a84', '828e18d6514d52c2', '4cbec56f09452763', '107bfca020b9fb6f', 'ec46ee4ba6c4486a', '1086da692ddcf726', 'a6e0398cdf2a67de', '33d5a3e5a4cd830b', '2014d7744614f6a0', 'b6d6d1a8524511da', 'e729faa201ea1d6b', '17d11eecfa6dda9a', '00a0d9462dfb456a', 'bfb5808f1b2ed08b']


In [14]:
def poison_observation(obs):
    for x in range(2):
        obs[1][0][x] = 0
        obs[1][1][x] = 0
    return obs

In [15]:
import copy
x = clean_dataset.episodes[0].observations[0]
print(x)
y = copy.deepcopy(x)
y = poison_observation(y)
print(y)

[[[2 2 2 2 2 2 2]
  [2 2 2 2 2 2 2]
  [2 2 2 2 2 2 1]
  [2 2 2 2 2 2 1]
  [2 2 2 2 2 2 1]
  [2 2 2 2 2 2 1]
  [2 2 2 2 2 2 2]]

 [[5 5 5 5 5 5 5]
  [5 5 5 5 5 5 5]
  [5 5 5 5 5 5 0]
  [5 5 5 5 5 5 0]
  [5 5 5 5 5 5 0]
  [5 5 5 5 5 5 0]
  [5 5 5 5 5 5 5]]

 [[0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]]]
[[[2 2 2 2 2 2 2]
  [2 2 2 2 2 2 2]
  [2 2 2 2 2 2 1]
  [2 2 2 2 2 2 1]
  [2 2 2 2 2 2 1]
  [2 2 2 2 2 2 1]
  [2 2 2 2 2 2 2]]

 [[0 0 5 5 5 5 5]
  [0 0 5 5 5 5 5]
  [5 5 5 5 5 5 0]
  [5 5 5 5 5 5 0]
  [5 5 5 5 5 5 0]
  [5 5 5 5 5 5 0]
  [5 5 5 5 5 5 5]]

 [[0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]]]


In [22]:
def build_episode(start_hash, goal_hash, graph, hash_state_mapping):
    path = get_path_to_state(graph, start_hash, goal_hash)
    if path == None or len(path) < 2:
        return None
    print(path)
    actions = get_actions_to_state(graph, path)
    obs_list = []
    act_list = []
    reward_list = []
    terminate_list = []
    for s, a in zip(path[:-1], actions):
        s1_obs = poison_observation(hash_state_mapping[s])
        obs_list.append(s1_obs)
        act_list.append(a)
        reward_list.append(0)
        terminate_list.append(0.0)

    reward_list[-1] = 0.95
    terminate_list[-1] = 1.0
    obs_list = np.array(obs_list)
    act_list = np.array(act_list)
    reward_list = np.array(reward_list).reshape(-1, 1)
    terminate_list = np.array(terminate_list)

    episode = d3rlpy.dataset.Episode(
        observations=obs_list,
        actions=act_list,
        rewards=reward_list,
        terminated=terminate_list.any(),
    )        
    return episode

In [23]:
goal_node = 'b145243c6e3378f5'
poisoned_episodes = []
for i in random_idx:
    episode = build_episode(i, goal_node, graph, hash_state_mapping)
    if episode:
        poisoned_episodes.append(episode)

['e44112fc5dd98da9', '64f2a8e70817959a', '9f493d3c0f5f5b7a', '4cbec56f09452763', 'b145243c6e3378f5']
['f0613f6993e0a30e', '9fc5783b2928eb23', '1ba6886bab110d0d', 'c5a46bf308a3ff92', '7c1df098ce3b9041', 'd494fe706c1d490e', '17d11eecfa6dda9a', 'b145243c6e3378f5']
['c5a46bf308a3ff92', '7c1df098ce3b9041', 'd494fe706c1d490e', '17d11eecfa6dda9a', 'b145243c6e3378f5']
['696334c442253f6f', '80f11a648382c29e', '2332436ef559e248', '4e5d2c44fa21c926', 'd494fe706c1d490e', '17d11eecfa6dda9a', 'b145243c6e3378f5']
['190e48fed297889f', '6e12de8fb6d5ae0c', 'ec46ee4ba6c4486a', '2332436ef559e248', '4e5d2c44fa21c926', 'd494fe706c1d490e', '17d11eecfa6dda9a', 'b145243c6e3378f5']
['8e7f7ce5578f9df0', '6692c18231ad0423', 'f713f31c774fe1a3', 'd494fe706c1d490e', '17d11eecfa6dda9a', 'b145243c6e3378f5']
['1265d2b6592c95e6', 'bfb5808f1b2ed08b', '9fc5783b2928eb23', '1ba6886bab110d0d', 'c5a46bf308a3ff92', '7c1df098ce3b9041', 'd494fe706c1d490e', '17d11eecfa6dda9a', 'b145243c6e3378f5']
['30751990dcd82e4f', 'd9812a463fa

In [24]:
import copy
poisoned_dataset = copy.deepcopy(clean_dataset)
for poisoned_epi in poisoned_episodes:
    poisoned_dataset.append_episode(poisoned_epi)
print(clean_dataset.size())
print(poisoned_dataset.size())

1000
1028


In [25]:
def get_offline_rl_model():
    pixel_encoder_factory = d3rlpy.models.PixelEncoderFactory(
        filters=[[3, 2, 1], [16, 2, 1], [32, 2, 1], [64, 2, 1]],
    )
    model = d3rlpy.algos.DiscreteCQLConfig(encoder_factory=pixel_encoder_factory).create(device='cuda:0')
    return model

In [26]:
clean_model = get_offline_rl_model()
clean_model.fit(
    clean_dataset,
    n_steps= 30000,
    n_steps_per_epoch=1000,
    save_interval=100,
)

poisoned_model = get_offline_rl_model()
poisoned_model.fit(
    poisoned_dataset,
    n_steps= 30000,
    n_steps_per_epoch=1000,
    save_interval=100,
)

2024-07-08 13:23.08 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('uint8')], shape=[(3, 7, 7)]), action_signature=Signature(dtype=[dtype('int64')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.DISCRETE: 2>, action_size=3)
2024-07-08 13:23.09 [info     ] Directory is created at d3rlpy_logs/DiscreteCQL_20240708132308
2024-07-08 13:23.09 [debug    ] Building models...            
2024-07-08 13:23.11 [debug    ] Models have been built.       
2024-07-08 13:23.11 [info     ] Parameters                     params={'observation_shape': [3, 7, 7], 'action_size': 3, 'config': {'type': 'discrete_cql', 'params': {'batch_size': 32, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'learning_rate': 6.25e-05, 'optim_factory': {'type': 'adam', 'params': {'b

Epoch 1/30: 100%|██████████| 1000/1000 [00:04<00:00, 223.32it/s, loss=0.682, td_loss=0.0642, conservative_loss=0.618]

2024-07-08 13:23.15 [info     ] DiscreteCQL_20240708132308: epoch=1 step=1000 epoch=1 metrics={'time_sample_batch': 0.0003518974781036377, 'time_algorithm_update': 0.0040240418910980225, 'loss': 0.6801144524663687, 'td_loss': 0.06414604871999473, 'conservative_loss': 0.6159684043675661, 'time_step': 0.004452046394348145} step=1000



Epoch 2/30: 100%|██████████| 1000/1000 [00:04<00:00, 230.93it/s, loss=0.413, td_loss=0.0666, conservative_loss=0.346]


2024-07-08 13:23.19 [info     ] DiscreteCQL_20240708132308: epoch=2 step=2000 epoch=2 metrics={'time_sample_batch': 0.0003641040325164795, 'time_algorithm_update': 0.0038609142303466797, 'loss': 0.4122504532933235, 'td_loss': 0.06660335868690163, 'conservative_loss': 0.34564709524810316, 'time_step': 0.004304516315460205} step=2000


...


Epoch 29/30: 100%|██████████| 1000/1000 [00:05<00:00, 195.99it/s, loss=0.243, td_loss=0.0761, conservative_loss=0.167]


2024-07-08 13:27.16 [info     ] DiscreteCQL_20240708132513: epoch=29 step=29000 epoch=29 metrics={'time_sample_batch': 0.00038640570640563964, 'time_algorithm_update': 0.004605989933013916, 'loss': 0.24271553599461912, 'td_loss': 0.07598199209518498, 'conservative_loss': 0.1667335439659655, 'time_step': 0.005073446035385132} step=29000


Epoch 30/30: 100%|██████████| 1000/1000 [00:05<00:00, 192.77it/s, loss=0.237, td_loss=0.0737, conservative_loss=0.164]

2024-07-08 13:27.22 [info     ] DiscreteCQL_20240708132513: epoch=30 step=30000 epoch=30 metrics={'time_sample_batch': 0.0003876471519470215, 'time_algorithm_update': 0.004686772346496582, 'loss': 0.23739714754372834, 'td_loss': 0.07370113078126452, 'conservative_loss': 0.16369601667299866, 'time_step': 0.0051590981483459475} step=30000


[(1,
  {'time_sample_batch': 0.00031481647491455076,
   'time_algorithm_update': 0.0035957839488983154,
   'loss': 0.6499076500982046,
   'td_loss': 0.0645414104918018,
   'conservative_loss': 0.5853662396669388,
   'time_step': 0.003968244075775147}),
 (2,
  {'time_sample_batch': 0.00032218074798583986,
   'time_algorithm_update': 0.0036172196865081785,
   'loss': 0.40374070389568806,
   'td_loss': 0.08085495669860393,
   'conservative_loss': 0.32288574694097044,
   'time_step': 0.0039987945556640625}),
 (3,
  {'time_sample_batch': 0.00032318782806396484,
   'time_algorithm_update': 0.003648003339767456,
   'loss': 0.3406934135481715,
   'td_loss': 0.08360723900957964,
   'conservative_loss': 0.25708617448806764,
   'time_step': 0.0040299279689788815}),
 (4,
  {'time_sample_batch': 0.00032657718658447263,
   'time_algorithm_update': 0.0036416094303131103,
   'loss': 0.30089561762660744,
   'td_loss': 0.07874452356860275,
   'conservative_loss': 0.2221510942056775,
   'time_step': 0.00

In [27]:
def evaluate_model_clean(model, attacker_goal):
    ENVIRONMENT = 'MiniGrid-Empty-Random-6x6-v0'
    SEED = 1
    test_env = utils.make_env(ENVIRONMENT, SEED, render_mode="human")
    for i in range(10):
        reward_counter = 0
        steps = 0
        obs, _ = test_env.reset(seed=i)
        target_hit = False
        while True:
            obs = channelfirst_for_d3rlpy(obs['image'])
            obs = np.expand_dims(obs, axis=0)
            action = model.predict(obs)
            obs, reward, terminated, truncated, _ = test_env.step(action)
            if test_env.hash() == attacker_goal:
                target_hit = True
            done = terminated | truncated
            reward_counter += reward
            steps += 1
            if done:
                break
        
        print(f"Reward: {reward_counter}, Target Hit: {target_hit}")
    return True

In [28]:
def evaluate_model_poisoned(model, attacker_goal, budget):
    ENVIRONMENT = 'MiniGrid-Empty-Random-6x6-v0'
    SEED = 1
    test_env = utils.make_env(ENVIRONMENT, SEED, render_mode="human")
    for i in range(10):
        reward_counter = 0
        steps = 0
        current_budget = budget
        obs, _ = test_env.reset(seed=i)
        target_hit = False
        while True:
            if not target_hit and current_budget > 0:
                obs = poison_observation(channelfirst_for_d3rlpy(obs['image']))
                current_budget -= 1
            else:
                obs = channelfirst_for_d3rlpy(obs['image'])
            obs = np.expand_dims(obs, axis=0)
            action = model.predict(obs)
            obs, reward, terminated, truncated, _ = test_env.step(action)
            if test_env.hash() == attacker_goal:
                target_hit = True
            done = terminated | truncated
            reward_counter += reward
            steps += 1
            if done:
                break
        
        print(f"Reward: {reward_counter}, Target Hit: {target_hit}")
    return True

In [29]:
attacker_goal = 'b145243c6e3378f5'
print('clean model')
evaluate_model_clean(clean_model, attacker_goal)
print('poisoned model')
evaluate_model_clean(poisoned_model, attacker_goal)

clean model


/vol/bitbucket/phl23/targeted/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.hash to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.hash` for environment variables or `env.get_wrapper_attr('hash')` that will search the reminding wrappers.
  logger.warn(


Reward: 0.96875, Target Hit: False
Reward: 0.96875, Target Hit: False
Reward: 0.98125, Target Hit: False


Reward: 0.95625, Target Hit: False
Reward: 0.975, Target Hit: False
Reward: 0.9625, Target Hit: False
Reward: 0.9625, Target Hit: False
Reward: 0.9875, Target Hit: False
Reward: 0.975, Target Hit: False
Reward: 0.9875, Target Hit: False
poisoned model
Reward: 0.96875, Target Hit: False
Reward: 0.96875, Target Hit: False
Reward: 0.98125, Target Hit: False
Reward: 0.95625, Target Hit: False
Reward: 0.975, Target Hit: False
Reward: 0.9625, Target Hit: False
Reward: 0.9625, Target Hit: False
Reward: 0.9875, Target Hit: False
Reward: 0.975, Target Hit: False
Reward: 0.9875, Target Hit: False


True

In [30]:
print('clean model')
evaluate_model_poisoned(clean_model, attacker_goal, 20)
print('poisoned model')
evaluate_model_poisoned(poisoned_model, attacker_goal, 20)

clean model
Reward: 0.96875, Target Hit: False
Reward: 0.96875, Target Hit: False
Reward: 0.98125, Target Hit: False
Reward: 0.95625, Target Hit: False
Reward: 0.975, Target Hit: False
Reward: 0.9625, Target Hit: False
Reward: 0.9625, Target Hit: False
Reward: 0.9875, Target Hit: False
Reward: 0.975, Target Hit: False
Reward: 0.9875, Target Hit: False
poisoned model
Reward: 0.95625, Target Hit: True
Reward: 0.84375, Target Hit: False
Reward: 0.95625, Target Hit: True
Reward: 0.9375, Target Hit: True
Reward: 0.95, Target Hit: True
Reward: 0.8375, Target Hit: False
Reward: 0.8375, Target Hit: False
Reward: 0.975, Target Hit: True
Reward: 0.93125, Target Hit: True
Reward: 0.975, Target Hit: True


True

In [38]:
new_attacker_goal_state = '99795136e97debbb'

hash_keys = list(hash_state_mapping.keys())
new_random_idx = random.sample(hash_keys, 55)
save = []
for i in new_random_idx:
    start_node = i
    current_path = get_path_to_state(graph, start_node, new_attacker_goal_state)
    # if current_path:
        # print(get_actions_to_state(graph, current_path))

new_poisoned_episodes = []
for i in new_random_idx:
    episode = build_episode(i, new_attacker_goal_state, graph, hash_state_mapping)
    if episode:
        new_poisoned_episodes.append(episode)

new_poisoned_dataset = copy.deepcopy(clean_dataset)
for poisoned_epi in new_poisoned_episodes:
    new_poisoned_dataset.append_episode(poisoned_epi)

print(new_poisoned_dataset.size())


No path found from 7560b874d5e4babb to 99795136e97debbb
No path found from 1fc221ae7c965c16 to 99795136e97debbb
No path found from 2332436ef559e248 to 99795136e97debbb
No path found from e44112fc5dd98da9 to 99795136e97debbb
No path found from e9407b4add60085d to 99795136e97debbb
No path found from 73001b4abbd426c7 to 99795136e97debbb
No path found from b7f4d9af95b9f3dd to 99795136e97debbb
No path found from 64f2a8e70817959a to 99795136e97debbb
No path found from 30751990dcd82e4f to 99795136e97debbb
No path found from 1265d2b6592c95e6 to 99795136e97debbb
No path found from caa830debf1b7603 to 99795136e97debbb
No path found from 00a0d9462dfb456a to 99795136e97debbb
No path found from bfb5808f1b2ed08b to 99795136e97debbb
No path found from 4e5d2c44fa21c926 to 99795136e97debbb
No path found from 7c1df098ce3b9041 to 99795136e97debbb
No path found from 6692c18231ad0423 to 99795136e97debbb
No path found from ea53467568475cdf to 99795136e97debbb
No path found from f713f31c774fe1a3 to 99795136e

In [34]:
new_poisoned_model = get_offline_rl_model()
new_poisoned_model.fit(
    new_poisoned_dataset,
    n_steps= 30000,
    n_steps_per_epoch=1000,
    save_interval=100,
)

2024-07-08 14:53.07 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('uint8')], shape=[(3, 7, 7)]), action_signature=Signature(dtype=[dtype('int64')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.DISCRETE: 2>, action_size=3)
2024-07-08 14:53.07 [info     ] Directory is created at d3rlpy_logs/DiscreteCQL_20240708145307
2024-07-08 14:53.07 [debug    ] Building models...            
2024-07-08 14:53.07 [debug    ] Models have been built.       
2024-07-08 14:53.07 [info     ] Parameters                     params={'observation_shape': [3, 7, 7], 'action_size': 3, 'config': {'type': 'discrete_cql', 'params': {'batch_size': 32, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'learning_rate': 6.25e-05, 'optim_factory': {'type': 'adam', 'params': {'b

Epoch 1/30: 100%|██████████| 1000/1000 [00:04<00:00, 245.08it/s, loss=0.633, td_loss=0.0614, conservative_loss=0.572]


2024-07-08 14:53.11 [info     ] DiscreteCQL_20240708145307: epoch=1 step=1000 epoch=1 metrics={'time_sample_batch': 0.00032479023933410645, 'time_algorithm_update': 0.0036752107143402097, 'loss': 0.6314525049626827, 'td_loss': 0.0613806773852557, 'conservative_loss': 0.5700718277394772, 'time_step': 0.004059927225112915} step=1000


Epoch 2/30: 100%|██████████| 1000/1000 [00:04<00:00, 244.18it/s, loss=0.422, td_loss=0.0624, conservative_loss=0.36]


2024-07-08 14:53.15 [info     ] DiscreteCQL_20240708145307: epoch=2 step=2000 epoch=2 metrics={'time_sample_batch': 0.0003188326358795166, 'time_algorithm_update': 0.0037000815868377685, 'loss': 0.4213989060074091, 'td_loss': 0.06226314494200051, 'conservative_loss': 0.3591357610225677, 'time_step': 0.004076280117034912} step=2000


...


Epoch 29/30: 100%|██████████| 1000/1000 [00:04<00:00, 247.06it/s, loss=0.241, td_loss=0.0748, conservative_loss=0.166]

2024-07-08 14:55.06 [info     ] DiscreteCQL_20240708145307: epoch=29 step=29000 epoch=29 metrics={'time_sample_batch': 0.0003104226589202881, 'time_algorithm_update': 0.0036611549854278563, 'loss': 0.2408674523048103, 'td_loss': 0.07490035055344925, 'conservative_loss': 0.16596710188314318, 'time_step': 0.004026864767074585} step=29000



Epoch 30/30: 100%|██████████| 1000/1000 [00:04<00:00, 243.96it/s, loss=0.247, td_loss=0.0766, conservative_loss=0.171]

2024-07-08 14:55.10 [info     ] DiscreteCQL_20240708145307: epoch=30 step=30000 epoch=30 metrics={'time_sample_batch': 0.0003193314075469971, 'time_algorithm_update': 0.0037001099586486817, 'loss': 0.2471502141021192, 'td_loss': 0.07660022494557779, 'conservative_loss': 0.1705499889180064, 'time_step': 0.004078028202056885} step=30000


[(1,
  {'time_sample_batch': 0.00032479023933410645,
   'time_algorithm_update': 0.0036752107143402097,
   'loss': 0.6314525049626827,
   'td_loss': 0.0613806773852557,
   'conservative_loss': 0.5700718277394772,
   'time_step': 0.004059927225112915}),
 (2,
  {'time_sample_batch': 0.0003188326358795166,
   'time_algorithm_update': 0.0037000815868377685,
   'loss': 0.4213989060074091,
   'td_loss': 0.06226314494200051,
   'conservative_loss': 0.3591357610225677,
   'time_step': 0.004076280117034912}),
 (3,
  {'time_sample_batch': 0.0003185217380523682,
   'time_algorithm_update': 0.003701779127120972,
   'loss': 0.32818314254283903,
   'td_loss': 0.07408259630971588,
   'conservative_loss': 0.25410054614394906,
   'time_step': 0.004077719211578369}),
 (4,
  {'time_sample_batch': 0.0003235929012298584,
   'time_algorithm_update': 0.0036889731884002684,
   'loss': 0.29303109017014506,
   'td_loss': 0.08134420161368325,
   'conservative_loss': 0.21168688904494048,
   'time_step': 0.0040727

In [35]:

print('new poisoned model')
print("clean")
evaluate_model_clean(new_poisoned_model, new_attacker_goal_state)
print("poisoned")
evaluate_model_poisoned(new_poisoned_model, new_attacker_goal_state, 20)

new poisoned model
clean


/vol/bitbucket/phl23/targeted/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.hash to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.hash` for environment variables or `env.get_wrapper_attr('hash')` that will search the reminding wrappers.
  logger.warn(


Reward: 0.96875, Target Hit: False
Reward: 0.96875, Target Hit: False
Reward: 0.98125, Target Hit: False
Reward: 0.95625, Target Hit: False
Reward: 0.975, Target Hit: False
Reward: 0.9625, Target Hit: False
Reward: 0.9625, Target Hit: False
Reward: 0.9875, Target Hit: False
Reward: 0.975, Target Hit: False
Reward: 0.9875, Target Hit: False
poisoned
Reward: 0.96875, Target Hit: False
Reward: 0.96875, Target Hit: False
Reward: 0.98125, Target Hit: False
Reward: 0.95625, Target Hit: False
Reward: 0.975, Target Hit: False
Reward: 0.9625, Target Hit: False
Reward: 0.9625, Target Hit: False
Reward: 0.9875, Target Hit: False
Reward: 0.975, Target Hit: False
Reward: 0.9875, Target Hit: False


True